In [1]:
import requests
import pandas as pd
import math
import aiohttp
import httpx
import asyncio
import ujson as json
from datetime import datetime
import time
import os
import sys
from typing import List, Optional, Dict, TypeAlias
JSON: TypeAlias = dict[str, "JSON"] | list["JSON"] | str | int | float | bool | None

from dotenv import load_dotenv
load_dotenv()

from CUSIP_Curve import CUSIP_Curve

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

In [3]:
def cookie_string_to_dict(cookie_string):
    cookie_pairs = cookie_string.split('; ')
    cookie_dict = {pair.split('=')[0]: pair.split('=')[1] for pair in cookie_pairs if '=' in pair}
    return cookie_dict

In [60]:
t1 = time.time()
cookie_headers = {
    "authority": "services-dynarep.ddwa.finra.org",
    "method": "OPTIONS",
    "path": "/public/reporting/v2/data/group/FixedIncomeMarket/name/TreasuryTradeHistory",
    "scheme": "https",
    "accept": "*/*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9",
    "access-control-request-headers": "content-type,x-xsrf-token",
    "access-control-request-method": "POST",
    "cache-control": "no-cache",
    "origin": "https://www.finra.org",
    "pragma": "no-cache",
    "priority": "u=1, i",
    "referer": "https://www.finra.org/",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
}

cookie_url = "https://services-dynarep.ddwa.finra.org/public/reporting/v2/group/Firm/name/ActiveIndividual/dynamiclookup/examCode" 
cookie_response = requests.get(cookie_url, headers=cookie_headers)
cookie_str = dict(cookie_response.headers)["set-cookie"]
cookie_dict = cookie_string_to_dict(cookie_string=cookie_str)
print("cookie fetch time (sec): ", time.time() - t1)

t1 = time.time()
headers = {
    "authority": "services-dynarep.ddwa.finra.org",
    "method": "POST",
    "path": "/public/reporting/v2/data/group/FixedIncomeMarket/name/TreasuryTradeHistory",
    "scheme": "https",
    "accept": "application/json, text/plain, */*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "no-cache",
    # "content-length": str(sys.maxsize),
    "content-type": "application/json",
    "dnt": "1",
    "origin": "https://www.finra.org",
    "pragma": "no-cache",
    "priority": "u=1, i",
    "referer": "https://www.finra.org/",
    "sec-ch-ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    "x-xsrf-token": cookie_dict["XSRF-TOKEN"],
    "cookie": cookie_str 
}

payload = {
    "fields": [
        "issueSymbolIdentifier",
        "cusip",
        "tradeDate",
        "tradeTime",
        "reportedTradeVolume",
        "priceType",
        "lastSalePrice",
        "lastSaleYield",
        "reportingSideCode",
        "contraPartyTypeCode",
    ],
    "dateRangeFilters": [
        {
            "fieldName": "tradeDate", 
            "startDate": "2024-07-31", 
            "endDate": "2024-07-31"
        },
    ],
    "compareFilters": [
        {
            "fieldName": "cusip",
            "fieldValue": "912810UA4",
            "compareType": "EQUAL"
        },
        # {
        #     "fieldName": "tradeTime",
        #     "compareType": "GTE",
        #     "fieldValue": "08:00:00"
        # },
        # {
        #     "fieldName": "tradeTime",
        #     "compareType": "LTE",
        #     "fieldValue": "09:00:00"
        # }
    ],
    "limit": 5000,
    "offset": 50000,
    # "recordMaxLimit": sys.maxsize,
    # "responsePayloadMaxSize": "100mb",
    # "sortFields": ["-tradeTime"],
}

url = "https://services-dynarep.ddwa.finra.org/public/reporting/v2/data/group/FixedIncomeMarket/name/TreasuryTradeHistory"
res = requests.post(url, json=payload, headers=headers)
if res.ok:
    print(json.dumps(res.json()["returnBody"]["headers"], indent=4))
    trade_data_json = json.loads(res.json()["returnBody"]["data"])
    df = pd.DataFrame(trade_data_json)
    display(df)
    # df.to_excel("temp.xlsx")
else:
    print("res status: ", res.status_code)

print("data fetch time (sec): ", time.time() - t1)

cookie fetch time (sec):  0.5769882202148438
{
    "Record-Max-Limit": [
        "5000"
    ],
    "Transfer-Encoding": [
        "chunked"
    ],
    "Connection": [
        "keep-alive"
    ],
    "Pragma": [
        "no-cache"
    ],
    "Record-Limit": [
        "5000"
    ],
    "Date": [
        "Tue, 13 Aug 2024 21:44:55 GMT"
    ],
    "Record-Total": [
        "54649"
    ],
    "X-Frame-Options": [
        "DENY"
    ],
    "FINRA-api-request-id": [
        "0550f79f-08e9-4e97-8ec8-ad46bfc76258"
    ],
    "Response-Payload-Max-Size": [
        "3mb"
    ],
    "Record-Offset": [
        "50000"
    ],
    "Cache-Control": [
        "no-cache, no-store, max-age=0, must-revalidate"
    ],
    "Expires": [
        "0"
    ],
    "data-version": [
        "1"
    ],
    "Content-Type": [
        "application\/json"
    ]
}


,tradeTime,cusip,issueSymbolIdentifier,priceType,reportingSideCode,productSubTypeCode,contraPartyTypeCode,tradeDate,reportedTradeVolume,lastSaleYield,lastSalePrice
0,10:39:32,912810UA4,TSRYS5802849,D,S,NOTE,D,2024-07-31,1000000.00,4.348796,104.582031
1,10:39:32,912810UA4,TSRYS5802849,D,S,NOTE,D,2024-07-31,1000000.00,4.345381,104.640625
2,10:39:32,912810UA4,TSRYS5802849,D,B,NOTE,C,2024-07-31,1000000.00,4.345381,104.640625
3,10:39:32,912810UA4,TSRYS5802849,D,S,NOTE,D,2024-07-31,1000000.00,4.345381,104.640625
4,10:39:32,912810UA4,TSRYS5802849,D,B,NOTE,C,2024-07-31,1000000.00,4.345381,104.640625
...,...,...,...,...,...,...,...,...,...,...,...
4644,11:54:58,912810UA4,TSRYS5802849,D,S,NOTE,C,2024-07-31,1000000.00,4.347193,104.609525
4645,11:54:58,912810UA4,TSRYS5802849,D,S,NOTE,D,2024-07-31,3000000.00,4.347202,104.609375
4646,11:55:01,912810UA4,TSRYS5802849,D,S,NOTE,D,2024-07-31,1000000.00,4.347885,104.597656
4647,11:55:01,912810UA4,TSRYS5802849,D,S,NOTE,T,2024-07-31,1000000.00,4.347885,104.597656


data fetch time (sec):  11.938526630401611


In [9]:
def temp_tester_httpx(start, end, cusips):
    async def build_tasks(client: httpx.AsyncClient, start, end, cusips):
        tasks = await CUSIP_Curve._build_fetch_tasks_historical_trace_data(client=client, cusips=cusips, start_date=start, end_date=end)
        return await asyncio.gather(*tasks)
    
    async def run_fetch_all(start, end, cusips):
        async with httpx.AsyncClient() as client:
            all_data = await build_tasks(client=client, cusips=cusips, start=start, end=end)
            return all_data

    results = asyncio.run(run_fetch_all(cusips=cusips, start=start, end=end))
    return results

def temp_tester_aio(start, end, cusips):
    async def build_tasks(session: aiohttp.ClientSession, start, end, cusips):
        tasks = await CUSIP_Curve._build_fetch_tasks_historical_trace_data(session=session, cusips=cusips, start_date=start, end_date=end)
        return await asyncio.gather(*tasks)
    
    async def run_fetch_all(start, end, cusips):
        async with aiohttp.ClientSession() as session:
            all_data = await build_tasks(session=session, cusips=cusips, start=start, end=end)
            return all_data

    results = asyncio.run(run_fetch_all(cusips=cusips, start=start, end=end))
    return results

In [ ]:
# CUSIP_Curve(debug_verbose=True).fetch_historcal_trace_trade_history_by_cusip(
#     cusips=["912810UA4"],
#     start_date=datetime(2024, 7, 29),
#     end_date=datetime(2024, 8, 2),
#     # xlsx_path="trace_trade_history_may54s.xlsx",
# )

In [3]:
df = CUSIP_Curve(no_logs_plz=True).build_curve_set(as_of_date=datetime(2024, 8, 12))
df.to_excel("temp.xlsx", index=False)
df

,cusip,security_type,auction_date,issue_date,maturity_date,price_per100,allocation_pctage,avg_med_yield,bid_to_cover_ratio,comp_accepted,...,changeFromPriorYear,outstanding_amt,portion_unstripped_amt,portion_stripped_amt,reconstituted_amt,mid_price,offer_yield,bid_yield,eod_yield,mid_yield
0,912796ZV4,Bill,2023-12-26,2023-12-28,2024-12-26,95.353944,17.810000,null,3.270000,42567207000,...,NaN,NaN,NaN,NaN,NaN,98.171555,5.055400,5.076792,5.017418,5.066096
1,912797GK7,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,-1.000000
2,912797GL5,Bill,2023-09-05,2023-09-07,2024-09-05,94.823111,81.610000,null,3.140000,40583323300,...,NaN,NaN,NaN,NaN,NaN,99.649666,5.689239,5.700253,5.469650,5.694746
3,912797GW1,Bill,2023-10-03,2023-10-05,2024-10-03,94.757389,36.110000,null,3.210000,42173646900,...,NaN,NaN,NaN,NaN,NaN,99.255389,5.430881,5.441605,5.345705,5.436243
4,912797HE0,Bill,2023-10-31,2023-11-02,2024-10-31,94.807944,31.870000,null,2.890000,42461266500,...,NaN,NaN,NaN,NaN,NaN,98.872222,5.348322,5.359071,5.269602,5.353697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,91282CLB5,Note,2024-07-23,2024-07-31,2026-07-31,99.888261,37.330000,4.393000,2.810000,68226981000,...,3.804444e+09,72783783.6,72783783.6,0.0,0.0,100.656250,4.022970,4.022970,4.022970,4.022970
397,91282CLC3,Note,2024-07-24,2024-07-31,2029-07-31,99.458270,96.280000,4.050000,2.400000,69796241000,...,3.859581e+09,73859635.1,73859635.1,0.0,0.0,101.062500,3.762965,3.762965,3.749130,3.762965
398,91282CLD1,Note,2024-07-25,2024-07-31,2031-07-31,99.777306,68.690000,4.111000,2.640000,43919495000,...,2.426022e+09,46426247.2,46426247.2,0.0,0.0,101.882812,3.813006,3.815555,3.800269,3.814281
399,91282CLF6,Note,2024-08-07,2024-08-15,2034-08-15,99.303721,91.610000,3.888000,2.320000,41874806900,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,-1.000000
